# UppASD interface

## Quick start: one temperature – notebook 1

In [ ]:
# create input objects
sim = MonteCarloSimulation()


# run single simulation (single API call)
sim.run(T=10, initmag=1, out='CrNiP')
# on disk:
# CrNiP/
#    drive-0/
sim.run(T=10, initmag=1, out='CrNiP')
# on disk:
# CrNiP/
#    run-0/
#    run-1/

Data('CrNiP')[0].Ms
Data('CrNiP')[0].restart_file

# USER RESPONSIBILITY
sim.run(T=10, initmag=1, out='CrNiP')
# on disk:
# CrNiP
sim.run(T=10, initmag=1, out='CrNiP')
RuntimeError("CrNiP exists")

# process simulation output
# - M at the given T

extract_parameters('CrNiP').Ms

Wishlist:
- running twice does not give an error
  - [NO to overwrite] overwrite (+ conceptionally simple if any existing folder is just deleted, - dangerous, potential loss of data) – error out would be better (in the notebook could delete existing data in the first cell)
  - save to different location (+ full provenance, - non-intuivite structure on disk, people need to learn the api)
  - [NO] run nothing, because it is already computed (difficult, guarantee identical simulation conditions, inherent randomness may lead to different states)

## Fit kuzmin – notebook 2
- Single temperature sweep, output: M(T)
- output: A(T), K(T), continuous M(T), optionally Tc

In [ ]:
# pass DFT output for jfile, momfile, posfile
jfile = Path(...)  # equivalent mom, pos

# create inpsd.dat, options: read some file, create from Python template/class
# create simulation object
# - needs to containt jfile, momfile, posfile
# - input parameters in some form

#something like
sim = MonteCarloSimulation.from_inpsd_file(jfile="./jfile", inpsd_dat="./inpsd.dat")
sim = Simulation(mode='M')
sim = Simulation(jfile=jfile, mode='S', init_mcsteps=2000)
sim = Simulation(jfile=jfile, inpsd_dat="./inpsd.dat") - NO



# define list of Ts
Ts = linspace(...)

# one API call to run full temperature sweep for Ts
# - control verbosity

# https://github.com/MaMMoS-project/management/issues/271, option 2b
drive0 = sim.temperature_sweep(T=[10, 100, 1000], restart_on_previous=True, initmag=1, out='CrNiP')
# on disk:
# CrNiP/
#    temperature_sweep-0/
#       run-0/
#       run-1/
#       run-2/
#       M(T)
drive1 = sim.temperature_sweep(T=[90, 100, 110], restart_on_previous=True, initmag=1, out='CrNiP')
# on disk:
# CrNiP/
#    temperature_sweep-0/
#       run-0/
#       run-1/
#       run-2/
#       M(T)
#    temperature_sweep-1/
#       run-0/
#       run-1/
#       run-2/
#       M(T)

In [ ]:
# read temperature sweep from disk and get one object back
# - could be returned from "one API call"
# - seperate call to read data

data = Data('CrNiP')
data.info
# table with details of simulations
# 1 line: temperature_drive-0/drive-{1-3}

# inspect its parameters, e.g. M(T)
temperature_data = data[0]
# output: Entity(ThermodynamicTemperature, [10, 20, 30], K)

In [ ]:
# pass T, M(T) to Kuzmin in mammos_analysis
mammos_analysis.kuzmin.kuzmin_properties(T=drive.T, Ms=drive.Ms, K1_0=1e5)  # K1_0 is optional

In [ ]:
drive = sim.run_temperature_sweep(T=Ts, restart_on_previous=True, initmag=temperature_data.sel(T=600, method="nearest").restart_file, out='CrNiP')
drive == data[1]

In [ ]:
mammos_analysis.kuzmin.kuzmin_properties(T=me.concat_flat(drive0.T, drive1.T), Ms=drive.Ms, K1_0=1e5)

In [ ]:
full_data = data[0].extend(data[1])
# OR
full_data = concat(data[0], data[1])

mammos_analysis.kuzmin.kuzmin_properties(T=full_data.T, Ms=full_data.Ms, K1_0=1e5)

In [ ]:
# https://github.com/MaMMoS-project/management/issues/271, option 1
sim.run_temperature_sweep(T=[10, 100, 1000], restart_on_previous=True, initmag=1, out='CrNiP')
# on disk:
# CrNiP/
#    drive-0/
#    drive-1/
#    drive-2/

sim.run_temperature_sweep(T=[90, 100, 110], restart_on_previous=True, initmag=1, out='CrNiP')
# on disk:
# CrNiP/
#    drive-0/
#    drive-1/
#    drive-2/
#    drive-3/
#    drive-4/
#    drive-5/

In [ ]:
data = Data('CrNiP')
data.info
# 6 lines: drive-[1-6]

T, Ms = extract_T_Ms(data[3:])

mammos_analysis.kuzmin.kuzmin_properties(T=T, Ms=Ms, K1_0=1e5)  # K1_0 is optional

In [ ]:
# user's responsibility
dirs = sim.run_temperature_sweep(T=[10, 100, 1000], restart_on_previous=True, initmag=1, out='CrNiP/T_{T}')
sim.run_temperature_sweep(T=[90, 100, 110], restart_on_previous=True, initmag='CrNiP/T_600', out='CrNiP_refined/T_{T}')

In [ ]:
T, Ms = extract_T_Ms([f'CrNiP/T_{T}' for T in [10, 100, 1000]], 'CrNiP_refined/T_*')

mammos_analysis.kuzmin.kuzmin_properties(T=T, Ms=Ms, K1_0=1e5)  # K1_0 is optional

## Different options to estimate Tc – notebook 3
- Specific heat
- Binder cumulant
- Two temperature sweeps

In [ ]:
# run worflow from cell 1 and 2
temperature_data = sim.run_temperature_sweep(T=[10, 100, 1000], restart_on_previous=True, initmag=1, out='CrNiP')


# inspect parameter Cv, peak in Cv
# - process peak using code from mammos_analysis
# - get an error estimate for Tc
Tc = mammos_analysis.Tc_from_Cv(T=temperature_data.T, Cv=temperature_data.Cv, [Ms=temperature_data.Ms])

# estimate Tc from Kuzmin fit as in cell 2
Tc = mammos_analysis.kuzmin.kuzmin_properties(T=temperature_data.T, Ms=temperature_data.Ms, K1_0=1e5).Tc

In [ ]:
# run second sweep for different system size
temperature_data_smaller = sim.run_temperature_sweep(T=[10, 100, 1000], restart_on_previous=True, initmag=1, out='CrNiP', system_size="smaller")

# one object per sweep: sweep1 and sweep2
# pass sweep1.binder_cumulant and sweep2.binder_cumulant to a function to estimate Tc

Tc = mammos_analysis.Tc_from_Binder_cumulant(temperature_data.Binder_cumulant, temperature_data_smaller.Binder_cumulant)

In [ ]:
# Analysis separate from calculation

data = Data('CrNiP')
Tc = mammos_analysis.Tc_from_Cv(T=data[0].T, Cv=data[0].Cv, [Ms=data[0].Ms])

Tc = mammos_analysis.Tc_from_Binder_cumulant(data[0].Binder_cumulant, data[1].Binder_cumulant)

In [ ]:
# table with value and error for the different Tc estimates

In [ ]:
# SAME FOR USER RESPONSIBILITY VERSION

In [ ]:
sim.run_temperature_sweep(T=[10, 100, 1000], restart_on_previous=True, initmag=1, out='CrNiP/T_{T}')

temperature_data = extract_parameters([f'CrNiP/T_{T}' for T in [10, 100, 1000]], 'CrNiP/T_*')
Tc = mammos_analysis.Tc_from_Cv(T=temperature_data.T, Cv=temperature_data.Cv, [Ms=temperature_data.Ms])
Tc = mammos_analysis.kuzmin.kuzmin_properties(T=temperature_data.T, Ms=temperature_data.Ms, K1_0=1e5).Tc

In [ ]:
sim.run_temperature_sweep(T=[10, 100, 1000], restart_on_previous=True, initmag=1, out='CrNiP_smaller/T_{T}')
temperature_data_smaller = extract_parameters([f'CrNiP_smaller/T_{T}' for T in [10, 100, 1000]], 'CrNiP_smaller/T_*')
Tc = mammos_analysis.Tc_from_Binder_cumulant(temperature_data.Binder_cumulant, temperature_data_smaller.Binder_cumulant)